In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision 
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid



In [ ]:
dta=((0.5,0.5,0.5),(0.5,0.5,0.5))
trans=transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize(*dta)
])

In [ ]:
ds=ImageFolder(root="/kaggle/input/animefacedataset/",transform=trans)

In [ ]:
batch_size=128
laten=100

In [ ]:
i,j=ds[0]
print(i.shape)

# DataLoader

dl=DataLoader(ds,batch_size=batch_size,shuffle=True,num_workers=2,pin_memory=True)
len(dl)

In [ ]:
img=next(iter(dl))
i,j=img
plt.figure(figsize=(60,60))
plt.imshow(make_grid(i,nrow=20).permute(1,2,0))

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main=nn.Sequential(
            nn.ConvTranspose2d(laten,512,padding=0,kernel_size=4,stride=1,bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(512,256,padding=1,kernel_size=4,stride=2,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(256,128,padding=1,kernel_size=4,stride=2,bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(128,64,padding=1,kernel_size=4,stride=2,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(64,3,padding=1,kernel_size=4,stride=2,bias=False),
            nn.Tanh()
            
            
        )
    def forward(self,x):
        return self.main(x)

In [ ]:
x=torch.rand(batch_size,laten,1,1)
gen=Generator()
op=gen(x)

In [ ]:
plt.figure(figsize=(30,30))
plt.imshow(make_grid(op.detach(),nrow=16).permute(1,2,0))

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main=nn.Sequential(
            nn.Conv2d(3,64,stride=2,kernel_size=4,padding=1,bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2,inplace=True),
            
            nn.Conv2d(64,128,stride=2,kernel_size=4,padding=1,bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2,inplace=True),
            
            nn.Conv2d(128,256,stride=2,kernel_size=4,padding=1,bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2,inplace=True),
            
            nn.Conv2d(256,512,stride=2,kernel_size=4,padding=1,bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2,inplace=True),
            
            nn.Conv2d(512,1,stride=1,kernel_size=4,padding=0,bias=False),
            nn.Flatten(),
            nn.Sigmoid()
            
        )
    def forward(self,x):
        return self.main(x)
        

In [ ]:
dis=Discriminator()
ri,j=next(iter(dl))
op=dis(ri)
print(op)



In [ ]:
op.shape

In [ ]:
from torchvision.utils import save_image
sample_dir = 'generated'
os.makedirs(sample_dir, exist_ok=True)
def save_samples(index, latent_tensors, show=True):
    fake_images = gen(latent_tensors)
    fake_fname = 'generated-images-{0:0=4d}.png'.format(index)
    save_image(fake_images, os.path.join(sample_dir, fake_fname), nrow=8)
    print('Saving', fake_fname)
    if show:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(fake_images.cpu().detach(), nrow=8).permute(1, 2, 0))

In [ ]:
fixed_latent = torch.randn(64, laten, 1, 1)
save_samples(0,fixed_latent)

# Training Functions

## discriminator

In [184]:
def fit_d(real,optim_d):
    optim_d.zero_grad()
    
#     real images
    rlabels=torch.ones(real.size(0),1)
    pred=dis(real)
    rloss=F.binary_cross_entropy(pred,rlabels)
    real_score = torch.mean(pred).item()
#     fake images generation

    fake_vector=torch.rand(batch_size,laten,1,1)
    fake_images=gen(fake_vector)
    flabels=torch.zeros(real.size(0),1)
    
#   train process

    pred=dis(fake_images)
    floss=F.binary_cross_entropy(pred,flabels)
    fake_score = torch.mean(pred).item()
    
    tloss=rloss+floss
    tloss.backward()
    
    optim_d.step()
    
    
    return tloss,real_score,fake_score
    
    
    

## Generator

In [185]:
def fit_g(optim_g):
    optim_g.zero_grad()
    
    img_vector=torch.rand(batch_size,laten,1,1)
    fake_image=gen(img_vector)
    
    lables=torch.ones(batch_size,1)
    
    pred=dis(fake_image)
    loss=F.binary_cross_entropy(pred,lables)
    loss.backward()
    optim_g.step()
    
    return loss.item()

In [186]:
from tqdm.notebook import tqdm
import torch.nn.functional as F

In [187]:
def fit(epoch,le,sidx=1):
    torch.cuda.empty_cache()
    
    opt_d = torch.optim.Adam(dis.parameters(), lr=lr, betas=(0.5, 0.999))
    opt_g = torch.optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))
    
    for i in range(epoch):
        for img,_ in dl:
            
            
            loss_d, real_score, fake_score = fit_d(img,opt_d)
            
            loss_g=fit_g(opt_g)
            
        save_samples(i+sidx, fixed_latent, show=False)
        print(f"genloss={losses_g},disloss={losses_d},rscore={real_scores}, fscore={fake_scores}")
    return losses_g, losses_d, real_scores, fake_scores

In [188]:
lr = 0.0002
epochs = 25

In [ ]:
fit(epochs,lr)